# Training mit Korrektur für Export

Zur Vollständigkeit hier nochmals das ganze Notebook, einfach erst unten mit der Korrektur.

Siehe [Errata:-Export-muss-anders-sein-als-bisher](#Errata:-Export-muss-anders-sein-als-bisher)

In [ ]:
from fastcore.foundation import L
from fastcore.xtras import Path  # @patch'd properties to the Pathlib module

from fastai.callback.schedule import fit_one_cycle, lr_find 

from fastai.data.block import CategoryBlock, DataBlock
from fastai.data.transforms import get_image_files, RandomSplitter

from fastai.learner import Learner

from fastai.metrics import error_rate
from torchvision.models.resnet import resnet34, resnet18
import torchvision.models as torch_models


from fastai.vision.all import (
    aug_transforms,
    ImageBlock,
    RegressionBlock,
    vision_learner,
    PILImage,
)

import json
import matplotlib.pyplot as plt

In [ ]:
image_path = Path('data/sluggish-cheetah')
image_path.exists()

In [ ]:
def read_data_from_json(image):
    with open(image_path/f'{image.stem}.json') as f:
        json_file = json.load(f)
        return[float(json_file['throttle']), float(json_file['steering'])]

In [ ]:
img = PILImage.create(next(image_path.glob('*.png')))
img.show()

In [ ]:
read_data_from_json(next(image_path.glob('*.png')))

In [ ]:
data_block = DataBlock(
    blocks=(ImageBlock, RegressionBlock(n_out=2)),
    get_items=get_image_files,
    splitter=RandomSplitter(),
    get_y=read_data_from_json
)

In [ ]:
data_loaders = data_block.dataloaders(Path(image_path), bs=32)

In [ ]:
data_loaders.show_batch()

In [ ]:
learn = vision_learner(data_loaders, resnet34)

In [ ]:
learn.lr_find()

In [ ]:
lr = 3e-03

In [ ]:
learn.fine_tune(25, lr)

In [ ]:
learn.show_results(ds_idx=1)

In [ ]:
learn.predict(next(image_path.glob('*.png')))

In [ ]:
files = get_image_files(image_path)
test_dl = learn.dls.test_dl(files)
preds = learn.get_preds(dl=test_dl)
preds

# Errata: Export muss anders sein als bisher

Neu müssen wir via ONNX gehen!

## ONNX installieren

In [ ]:
# onnx installieren
!pip install onnx --quiet

## Unsere Daten exportieren

* Der Pfad `models/example.onnx` ist was angepasst werden sollte, sonst wird der potentiell überschrieben.
* `input_names=["image"]` und `output_names=["data"]` müssen so heissen und dürfen nicht umbenannt werden. Kleine Frage: Warum könnte das sein?


In [ ]:
import torch
from PIL import Image
from torchvision import transforms


model = learn.model
# Modell auf eval setzen, damit die Gewichte korrekt exportiert werden!

model.eval()

# wir brauchen ein Beispiel Bild um den Input für ONNX festzulegen
convert_tensor = transforms.ToTensor()
with Image.open(next(image_path.glob('*.png'))) as img:
    # wir müssen das Bild in das richtige Format bringen, das
    # übernimmt fastai sonst für uns, onnx kennt aber fastai nicht, nur unser model.
    tensor_img = convert_tensor(img).unsqueeze(0)

torch.onnx.export(
    model,
    tensor_img.cuda(),
    "models/example.onnx",
    input_names=["image"],
    output_names=["data"]
)

## Exportiertes Modell auf Korrektheit prüfen

In [ ]:
# ensure correctness
import onnx

onnx_model = onnx.load("models/example.onnx")
onnx.checker.check_model(onnx_model)

## Durchspielen, was auf unserem Fahrzeug passieren wird

Dafür braucht es *nur* onnxruntime (`ort`) und pytorch.

In [ ]:
# onnxruntime installieren
!pip install onnxruntime --quiet

In [ ]:
from PIL import Image
from torchvision import transforms
from pathlib import Path
import onnxruntime as ort
import numpy

convert_tensor = transforms.ToTensor()

def get_inference(image_path):
    with Image.open(image_path) as img:
        # wir müssen das Bild in das richtige Format bringen, das
        # übernimmt fastai sonst für uns, onnx kennt aber fastai nicht, nur unser model.
        tensor_img = convert_tensor(img).unsqueeze(0).numpy()
    
    ort_sess = ort.InferenceSession('models/example.onnx')
    outputs = ort_sess.run(None, {'image': tensor_img})
    steering, throttle = outputs[0][0]
    return steering, throttle

Tut das auch?

In [ ]:
# wir nehmen ein einzelnes Bild von uns, auf dem Fahrzeug kommt dies vom Stream
example_image_path = next(Path('data/sluggish-cheetah').glob('*.png'))

steering, throttle = get_inference(example_image_path)
steering, throttle

### Wer mag, kann das auch mit ein paar Bildern prüfen

In [ ]:
images_iterator = Path('data/sluggish-cheetah').glob('*.png')

# Annahme: es gibt (mindestens) 20 Bilder
for _ in range(20):
    im = next(images_iterator)
    steering, throttle = get_inference(im)
    print(steering, throttle)

# Auf das Fahrzeug übertragen

**Wenn das funktioniert hat**, können wir das models/example.onnx herunterladen und auf dem Fahrzeug an zu den uploads kopieren (zB. `uploads/models/2022-11-11_round-course.onnx`) und dann `SELF_DRIVING_MODEL_PATH` im docker-compose.yml anpassen (zB. `SELF_DRIVING_MODEL_PATH: "uploads/models/2022-11-11_round-course.onnx"`) und `docker compose up -d` ausführen.
Wenn der Learner importiert wurde, kann dieser wie vorhin weiter verwendet werden.